# 使うライブラリ

In [1]:
!pip install git+https://github.com/pfnet-research/xfeat.git

  Cloning https://github.com/pfnet-research/xfeat.git to /tmp/pip-req-build-3w0c4rdq
  Running command git clone -q https://github.com/pfnet-research/xfeat.git /tmp/pip-req-build-3w0c4rdq


#import

In [2]:
import pandas as pd
import os
import plotly.express as px

from sklearn.model_selection import KFold
from functools import partial
import lightgbm as lgb
import optuna
from xfeat import SelectCategorical, LabelEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

# pandasで作成するプロットの種類をplotlyに変更
pd.options.plotting.backend = 'plotly'

In [3]:
cd /content/drive/MyDrive/signate/signate_PM

/content/drive/MyDrive/signate/signate_PM


In [4]:
ls

lgbm1.ipynb  README.md  submit_sample.csv  test.csv  train.csv


# data取り込み

In [46]:
train   =pd.read_csv('train.csv')
test   =pd.read_csv('test.csv')
sample =pd.read_csv('submit_sample.csv',names=['id','pm25_mid']) 

train["timestamp"] = pd.to_datetime(train[["year", "month", "day"]])
test["timestamp"] = pd.to_datetime(test[["year", "month", "day"]])

# EDA

In [6]:
train

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,...,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid,timestamp
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,38,0.749,...,1.088,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901,2019-01-01
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,47,2.594,...,3.473,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741,2019-01-01
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,17,1.190,...,2.107,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918,2019-01-01
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,63,4.586,...,0.503,3.592,2.485,116,7.146,10.685,13.344,9.417,174.370,2019-01-01
4,5,2019,1,1,Australia,Perth,-31.95224,115.86140,47,4.689,...,0.755,3.396,1.937,93,1.091,3.277,12.272,4.109,167.063,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195936,195937,2021,12,31,United States,Jacksonville,30.33218,-81.65565,12,0.694,...,2.710,6.125,3.757,12,16.774,22.679,26.058,13.252,16.150,2021-12-31
195937,195938,2021,12,31,United States,Las Vegas,36.17497,-115.13722,14,0.528,...,2.974,6.861,8.354,12,10.432,14.741,15.827,7.078,16.895,2021-12-31
195938,195939,2021,12,31,United States,Milwaukee,43.03890,-87.90647,171,1.975,...,1.087,2.578,0.612,26,2.049,3.531,6.686,5.286,86.299,2021-12-31
195939,195940,2021,12,31,Vietnam,Hanoi,21.02450,105.84117,31,2.613,...,3.058,6.005,6.085,51,1.922,7.443,7.716,4.642,36.523,2021-12-31


In [7]:
train.columns

Index(['id', 'year', 'month', 'day', 'Country', 'City', 'lat', 'lon', 'co_cnt',
       'co_min', 'co_mid', 'co_max', 'co_var', 'o3_cnt', 'o3_min', 'o3_mid',
       'o3_max', 'o3_var', 'so2_cnt', 'so2_min', 'so2_mid', 'so2_max',
       'so2_var', 'no2_cnt', 'no2_min', 'no2_mid', 'no2_max', 'no2_var',
       'temperature_cnt', 'temperature_min', 'temperature_mid',
       'temperature_max', 'temperature_var', 'humidity_cnt', 'humidity_min',
       'humidity_mid', 'humidity_max', 'humidity_var', 'pressure_cnt',
       'pressure_min', 'pressure_mid', 'pressure_max', 'pressure_var',
       'ws_cnt', 'ws_min', 'ws_mid', 'ws_max', 'ws_var', 'dew_cnt', 'dew_min',
       'dew_mid', 'dew_max', 'dew_var', 'pm25_mid', 'timestamp'],
      dtype='object')

In [8]:
train.dtypes

id                          int64
year                        int64
month                       int64
day                         int64
Country                    object
City                       object
lat                       float64
lon                       float64
co_cnt                      int64
co_min                    float64
co_mid                    float64
co_max                    float64
co_var                    float64
o3_cnt                      int64
o3_min                    float64
o3_mid                    float64
o3_max                    float64
o3_var                    float64
so2_cnt                     int64
so2_min                   float64
so2_mid                   float64
so2_max                   float64
so2_var                   float64
no2_cnt                     int64
no2_min                   float64
no2_mid                   float64
no2_max                   float64
no2_var                   float64
temperature_cnt             int64
temperature_mi

In [9]:
for i in train.columns:
  fig = px.histogram(train[i], x=i)
  fig.show()
  print(train[i].value_counts())
  print()

Output hidden; open in https://colab.research.google.com to view.

In [10]:
SelectCategorical().fit_transform(train)

,Country,City,timestamp
0,Australia,Brisbane,2019-01-01
1,Australia,Darwin,2019-01-01
2,Australia,Melbourne,2019-01-01
3,Australia,Newcastle,2019-01-01
4,Australia,Perth,2019-01-01
...,...,...,...
195936,United States,Jacksonville,2021-12-31
195937,United States,Las Vegas,2021-12-31
195938,United States,Milwaukee,2021-12-31
195939,Vietnam,Hanoi,2021-12-31


In [12]:
# fold = KFold(n_splits=30, shuffle=False)
# encoder = TargetEncoder(
#     input_cols=["timestamp"], 
#     target_col="pm25_mid",
#     fold=fold,
#     output_suffix="_re"
#     )

# train = encoder.fit_transform(train)
# test = encoder.fit_transform(test)
# train[["pm25_mid", "timestamp", "timestamp_re"]].head(12)

In [13]:
print(train['timestamp_re'].value_counts())
print()
len(train['timestamp'].unique())

0.000000     190585
41.769501       190
51.584751       186
41.347469       184
20.256800       174
52.827557       173
43.977333       171
53.328758       165
41.165577       164
47.259075       160
49.037785       160
41.090809       153
55.910316       153
42.715332       151
66.152321       149
56.584332       143
48.719509       143
58.111130       138
50.321907       137
60.321728       134
54.400818       122
54.213776       121
46.912254       111
49.475708       109
54.379658       108
52.351784       108
34.515533       105
66.324547       103
55.122166        99
52.636917        97
78.602402        91
48.606087        88
64.564217        82
52.145000        78
82.045578        68
64.196396        65
53.662853        63
49.696568        62
79.241882        61
62.502411        59
49.023903        58
66.762985        54
57.836624        47
52.915554        42
69.569588        40
65.601944        39
49.648430        36
76.614899        31
47.900398        27
49.479439        26


1095

In [14]:
# カラム指定したい場合は、pd.get_dummies(df_dummy, columns=['workclass'])
df = pd.get_dummies(train,columns=['Country','City'])
test = pd.get_dummies(test,columns=['Country','City'])
df

,id,year,month,day,lat,lon,co_cnt,co_min,co_mid,co_max,...,City_Yunfu,City_Zagreb,City_Zanjān,City_Zenica,City_Zhengzhou,City_Zhuzhou,City_Ürümqi,City_İzmit,City_Łódź,City_Ōita
0,1,2019,1,1,-27.46794,153.02809,38,0.749,2.590,2.633,...,0,0,0,0,0,0,0,0,0,0
1,2,2019,1,1,-12.46113,130.84185,47,2.594,3.181,4.828,...,0,0,0,0,0,0,0,0,0,0
2,3,2019,1,1,-37.81400,144.96332,17,1.190,1.197,2.200,...,0,0,0,0,0,0,0,0,0,0
3,4,2019,1,1,-32.92953,151.78010,63,4.586,11.044,14.802,...,0,0,0,0,0,0,0,0,0,0
4,5,2019,1,1,-31.95224,115.86140,47,4.689,8.681,11.100,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195936,195937,2021,12,31,30.33218,-81.65565,12,0.694,0.995,1.301,...,0,0,0,0,0,0,0,0,0,0
195937,195938,2021,12,31,36.17497,-115.13722,14,0.528,1.256,3.226,...,0,0,0,0,0,0,0,0,0,0
195938,195939,2021,12,31,43.03890,-87.90647,171,1.975,6.627,6.639,...,0,0,0,0,0,0,0,0,0,0
195939,195940,2021,12,31,21.02450,105.84117,31,2.613,2.704,8.767,...,0,0,0,0,0,0,0,0,0,0


In [15]:
def feature_engineering(df):
    cols = df.columns.tolist()

    encoder = Pipeline([
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="+",
                               r=2,
                               output_suffix="_plus"),
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="*",
                               r=2,
                               output_suffix="_mul"),
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="-",
                               r=2,
                               output_suffix="_minus"),
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="+",
                               r=3,
                               output_suffix="_plus"),
    ])
    return encoder.fit_transform(df)

In [16]:
df_num=SelectNumerical().fit_transform(df)
test_num=SelectNumerical().fit_transform(test)

In [17]:
# print("After adding interaction features:")
# df_num = feature_engineering(df_num)

In [20]:
LGBM_PARAMS = {
        "objective": "regression",
        "metric": "RMSE",
        "verbosity": -1,
}


def objective(df, selector, trial):
    selector.set_trial(trial)
    selector.fit(df)
    input_cols = selector.get_selected_cols()

    # Hyper Parameter Tuning するパラメータと範囲を設定する
    lgbm_params = {
        'num_leaves': trial.suggest_int("num_leaves", 3, 10),
        'max_depth': trial.suggest_int("max_depth", 3, 10),
    }
    lgbm_params.update(LGBM_PARAMS)

    # Evaluate with selected columns
    train_set = lgb.Dataset(df[input_cols], label=df["pm25_mid"])
    scores = lgb.cv(lgbm_params, train_set, num_boost_round=200, stratified=False, seed=1)
    
    rmse_score = scores['rmse-mean'][-1]
    return rmse_score


# 特徴量探索のための説明変数を設定する
# encoded_train_df は、No.7で作成したDataFrameと同じ
input_cols = list(df_num.columns)
input_cols.remove("pm25_mid")


# 特徴量探索器を作成する
selector = GBDTFeatureExplorer(
    input_cols=input_cols,
    target_col="pm25_mid",
    fit_once=True,
    threshold_range=(0.8, 1.0),
    lgbm_params=LGBM_PARAMS,
)

# Hyper Parameter Tuning を行う
study = optuna.create_study(direction="minimize")
study.optimize(partial(objective, df_num, selector), n_trials=10)

# 選択された特徴量を確認する
selector.from_trial(study.best_trial)
print("Selected columns:", selector.get_selected_cols())

[I 2022-05-01 09:34:48,692] A new study created in memory with name: no-name-f151f19c-97fa-4dde-afe7-84baafd06e02
[I 2022-05-01 09:35:29,033] Trial 0 finished with value: 21.42813844615826 and parameters: {'GBDTFeatureSelector.threshold': 0.9043829733591289, 'num_leaves': 8, 'max_depth': 5}. Best is trial 0 with value: 21.42813844615826.
[I 2022-05-01 09:35:48,438] Trial 1 finished with value: 23.297727381594267 and parameters: {'GBDTFeatureSelector.threshold': 0.9613448769361792, 'num_leaves': 3, 'max_depth': 4}. Best is trial 0 with value: 21.42813844615826.
[I 2022-05-01 09:36:14,688] Trial 2 finished with value: 21.594634795994274 and parameters: {'GBDTFeatureSelector.threshold': 0.8070211975363236, 'num_leaves': 7, 'max_depth': 6}. Best is trial 0 with value: 21.42813844615826.
[I 2022-05-01 09:36:38,695] Trial 3 finished with value: 21.909469067510585 and parameters: {'GBDTFeatureSelector.threshold': 0.8216631152535279, 'num_leaves': 6, 'max_depth': 4}. Best is trial 0 with value

Selected columns: ['co_mid', 'month', 'no2_mid', 'lon', 'no2_min', 'o3_mid', 'so2_mid', 'co_min', 'lat', 'so2_min', 'o3_max', 'o3_var', 'Country_India', 'co_var', 'temperature_mid', 'pressure_min', 'id', 'so2_cnt', 'pressure_max', 'so2_max', 'ws_min', 'temperature_max', 'humidity_min', 'co_max', 'ws_mid', 'humidity_mid', 'so2_var', 'no2_var', 'o3_min', 'Country_China', 'temperature_min', 'o3_cnt', 'pressure_cnt', 'humidity_cnt', 'dew_max', 'dew_mid', 'co_cnt', 'no2_max', 'temperature_cnt', 'no2_cnt', 'ws_var', 'dew_min', 'temperature_var', 'dew_cnt', 'ws_cnt', 'pressure_var', 'Country_Thailand', 'City_Shillong', 'ws_max', 'City_Chandigarh', 'humidity_max', 'City_Zhuzhou', 'City_Vereeniging', 'City_Chiang Mai', 'Country_Turkey', 'City_Samsun', 'City_Beijing', 'City_Hyderabad', 'City_Antakya', 'City_Lhasa', 'City_Lampang', 'City_Hāpur', 'Country_Poland', 'Country_Bosnia and Herzegovina', 'City_Guangzhou', 'City_Changsha', 'City_Toluca', 'City_Kolkata', 'City_Xinxiang', 'Country_South Afr

In [21]:
print(study.best_params)
print(study.best_value)

{'GBDTFeatureSelector.threshold': 0.9675435309982091, 'num_leaves': 9, 'max_depth': 9}
21.21629935394368


# モデル制作

In [22]:
target = df_num["pm25_mid"]
cols=selector.get_selected_cols()

df_train=df_num
df_test=test_num

In [30]:
for i in cols:
  print(i)
  if i not in df_test.columns:
    df_test[i]=0

co_mid
month
no2_mid
lon
no2_min
o3_mid
so2_mid
co_min
lat
so2_min
o3_max
o3_var
Country_India
co_var
temperature_mid
pressure_min
id
so2_cnt
pressure_max
so2_max
ws_min
temperature_max
humidity_min
co_max
ws_mid
humidity_mid
so2_var
no2_var
o3_min
Country_China
temperature_min
o3_cnt
pressure_cnt
humidity_cnt
dew_max
dew_mid
co_cnt
no2_max
temperature_cnt
no2_cnt
ws_var
dew_min
temperature_var
dew_cnt
ws_cnt
pressure_var
Country_Thailand
City_Shillong
ws_max
City_Chandigarh
humidity_max
City_Zhuzhou
City_Vereeniging
City_Chiang Mai
Country_Turkey
City_Samsun
City_Beijing
City_Hyderabad
City_Antakya
City_Lhasa
City_Lampang
City_Hāpur
Country_Poland
Country_Bosnia and Herzegovina
City_Guangzhou
City_Changsha
City_Toluca
City_Kolkata
City_Xinxiang
Country_South Africa
City_Erzurum
pressure_mid
City_Bhopal
City_Pretoria
City_Ürümqi
City_Talca
City_Hanoi
City_Tel Aviv
City_Konya
City_Lucknow
City_Ningbo
City_İzmit
City_Hạ Long
City_Yokohama
City_Thrissur
City_Istanbul
humidity_var
City_Zhe

In [31]:
# parameters

learning_rate = 0.10
SEED = 42
lgb_params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": learning_rate,
    "num_leaves": 8,
    "min_data_in_leaf": 40,
    #"colsample_bytree": 1.0,
    #"feature_fraction": 1.0,
    #"bagging_freq": 0,
    #"bagging_fraction": 1.0,
    "verbosity": -1,
    "seed": SEED,
}

In [32]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np

SEED_SKF = SEED
np.random.seed(SEED)
 
FOLD = 10
skf = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=SEED_SKF)
kf = KFold(n_splits=FOLD, shuffle=True, random_state=SEED)

oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
df_feature_importance = pd.DataFrame()

features = cols 

N_CLASSES=1

In [33]:
for fold_, (indexes_trn, indexes_val) in enumerate(kf.split(df_train.values, target.values)):
    print(f"------------------------------ fold {fold_} ------------------------------")
    
    df_trn = df_train.loc[indexes_trn].reset_index(drop=True)
    df_val = df_train.loc[indexes_val].reset_index(drop=True)
    target_trn = target.loc[indexes_trn].reset_index(drop=True)
    target_val = target.loc[indexes_val].reset_index(drop=True)
    
    
    lgb_train = lgb.Dataset(
        df_trn.loc[:, features],
        label=target_trn,
        feature_name=features,
    )
    lgb_valid = lgb.Dataset(
        df_val.loc[:, features],
        label=target_val,
        feature_name=features,
    )
    
    lgb_params["learning_rate"] = learning_rate + np.random.random() * 0.001  # おまじない
    num_round = 999999999
    model = lgb.train(
        lgb_params,
        lgb_train, 
        num_round, 
        valid_sets=[lgb_train, lgb_valid], 
        verbose_eval=300,
        early_stopping_rounds=300 if num_round >= 1e8 else None,
        fobj=None,
        #feval=lgb_metric,
    )
    
    # cv
    prediction_round = model.best_iteration+150 if num_round >= 1e8 else num_round  # おまじない
    oof[indexes_val] = model.predict(df_val[features], num_iteration=prediction_round) 
    
    # feature importance
    df_fold_importance = pd.DataFrame()
    df_fold_importance["feature"] = features
    df_fold_importance["importance"] = model.feature_importance()
    df_fold_importance["fold"] = fold_

    # prediction for test data
    predictions += model.predict(df_test[features], num_iteration=prediction_round) / FOLD
    print()

------------------------------ fold 0 ------------------------------
Training until validation scores don't improve for 300 rounds.
[300]	training's rmse: 20.674	valid_1's rmse: 20.8773
[600]	training's rmse: 19.9047	valid_1's rmse: 20.3768
[900]	training's rmse: 19.3578	valid_1's rmse: 20.0801
[1200]	training's rmse: 18.9305	valid_1's rmse: 19.8877
[1500]	training's rmse: 18.583	valid_1's rmse: 19.7545
[1800]	training's rmse: 18.2741	valid_1's rmse: 19.6501
[2100]	training's rmse: 17.982	valid_1's rmse: 19.5575
[2400]	training's rmse: 17.7174	valid_1's rmse: 19.4857
[2700]	training's rmse: 17.472	valid_1's rmse: 19.4347
[3000]	training's rmse: 17.2406	valid_1's rmse: 19.3905
[3300]	training's rmse: 17.0235	valid_1's rmse: 19.3636
[3600]	training's rmse: 16.8195	valid_1's rmse: 19.345
[3900]	training's rmse: 16.6266	valid_1's rmse: 19.3228
[4200]	training's rmse: 16.4372	valid_1's rmse: 19.3009
[4500]	training's rmse: 16.2533	valid_1's rmse: 19.281
[4800]	training's rmse: 16.0727	valid

In [34]:
pd.set_option('display.max_rows', 500)
df_fold_importance 

,feature,importance,fold
0,co_mid,1720,9
1,month,834,9
2,no2_mid,1445,9
3,lon,800,9
4,no2_min,1539,9
5,o3_mid,1113,9
6,so2_mid,1147,9
7,co_min,1253,9
8,lat,777,9
9,so2_min,968,9


In [35]:
from sklearn.metrics import mean_squared_error

df_test["prediction"] = predictions.tolist()
score=np.sqrt(mean_squared_error(target, oof))
print("CV score")
print(f"  rmse: {score:8.5f}")

CV score
  rmse: 19.20496


In [47]:
sample=sample.set_axis(['id','pm25_mid'],axis=1)

In [48]:
df_submission = sample.copy()
df_submission["pm25_mid"] = df_submission["id"].map(dict(df_test[["id", "prediction"]].values))
assert not df_submission["pm25_mid"].isna().any()
display(df_submission.head(10))

# make submission file
df_submission.to_csv("v1.csv",header=None,index=None)

,id,pm25_mid
0,195942,21.442720
1,195943,39.926963
2,195944,22.894570
3,195945,51.242715
4,195946,127.936115
5,195947,25.145314
6,195948,109.799183
7,195949,69.342227
8,195950,46.483877
9,195951,111.952983
